In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 200 kB 49.6 MB/s 
     |████████████████████████████████| 199 kB 10.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=691e461293c5320ad54924619cbf096045b8c70f4af45f8949f42c27bd677ae4
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.stat import Correlation
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.appName("Decision Tree").getOrCreate()

In [ ]:
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/IOT/Fish.csv", inferSchema = True, header = True)

In [ ]:
df.show()

+-------+------+-------+-------+-------+-------+------+
|Species|Weight|Length1|Length2|Length3| Height| Width|
+-------+------+-------+-------+-------+-------+------+
|  Bream| 242.0|   23.2|   25.4|   30.0|  11.52|  4.02|
|  Bream| 290.0|   24.0|   26.3|   31.2|  12.48|4.3056|
|  Bream| 340.0|   23.9|   26.5|   31.1|12.3778|4.6961|
|  Bream| 363.0|   26.3|   29.0|   33.5|  12.73|4.4555|
|  Bream| 430.0|   26.5|   29.0|   34.0| 12.444| 5.134|
|  Bream| 450.0|   26.8|   29.7|   34.7|13.6024|4.9274|
|  Bream| 500.0|   26.8|   29.7|   34.5|14.1795|5.2785|
|  Bream| 390.0|   27.6|   30.0|   35.0|  12.67|  4.69|
|  Bream| 450.0|   27.6|   30.0|   35.1|14.0049|4.8438|
|  Bream| 500.0|   28.5|   30.7|   36.2|14.2266|4.9594|
|  Bream| 475.0|   28.4|   31.0|   36.2|14.2628|5.1042|
|  Bream| 500.0|   28.7|   31.0|   36.2|14.3714|4.8146|
|  Bream| 500.0|   29.1|   31.5|   36.4|13.7592| 4.368|
|  Bream| 340.0|   29.5|   32.0|   37.3|13.9129|5.0728|
|  Bream| 600.0|   29.4|   32.0|   37.2|14.9544|

In [ ]:
df.count()

159

In [ ]:
len(df.columns)

7

In [ ]:
df.columns

['Species', 'Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

In [ ]:
df.printSchema()

root
 |-- Species: string (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Length1: double (nullable = true)
 |-- Length2: double (nullable = true)
 |-- Length3: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Width: double (nullable = true)



In [ ]:
df.describe().show()

+-------+---------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|summary|  Species|            Weight|          Length1|           Length2|           Length3|           Height|             Width|
+-------+---------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|  count|      159|               159|              159|               159|               159|              159|               159|
|   mean|     null| 398.3264150943396|26.24716981132075|28.415723270440253|31.227044025157248|8.970993710691822| 4.417485534591194|
| stddev|     null|357.97831655089306|9.996441210553128|10.716328098884247| 11.61024583269096|4.286207619968869|1.6858038699921665|
|    min|    Bream|               0.0|              7.5|               8.4|               8.8|           1.7284|            1.0476|
|    max|Whitefish|            1650.0|             59.0|              63.4| 

In [ ]:
df.head(5)

[Row(Species='Bream', Weight=242.0, Length1=23.2, Length2=25.4, Length3=30.0, Height=11.52, Width=4.02),
 Row(Species='Bream', Weight=290.0, Length1=24.0, Length2=26.3, Length3=31.2, Height=12.48, Width=4.3056),
 Row(Species='Bream', Weight=340.0, Length1=23.9, Length2=26.5, Length3=31.1, Height=12.3778, Width=4.6961),
 Row(Species='Bream', Weight=363.0, Length1=26.3, Length2=29.0, Length3=33.5, Height=12.73, Width=4.4555),
 Row(Species='Bream', Weight=430.0, Length1=26.5, Length2=29.0, Length3=34.0, Height=12.444, Width=5.134)]

In [ ]:
df.printSchema()

root
 |-- Species: string (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Length1: double (nullable = true)
 |-- Length2: double (nullable = true)
 |-- Length3: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Width: double (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol = "Species", outputCol = "Species_cat")

In [ ]:
indexed = indexer.fit(df).transform(df)

In [ ]:
indexed.show()

+-------+------+-------+-------+-------+-------+------+-----------+
|Species|Weight|Length1|Length2|Length3| Height| Width|Species_cat|
+-------+------+-------+-------+-------+-------+------+-----------+
|  Bream| 242.0|   23.2|   25.4|   30.0|  11.52|  4.02|        1.0|
|  Bream| 290.0|   24.0|   26.3|   31.2|  12.48|4.3056|        1.0|
|  Bream| 340.0|   23.9|   26.5|   31.1|12.3778|4.6961|        1.0|
|  Bream| 363.0|   26.3|   29.0|   33.5|  12.73|4.4555|        1.0|
|  Bream| 430.0|   26.5|   29.0|   34.0| 12.444| 5.134|        1.0|
|  Bream| 450.0|   26.8|   29.7|   34.7|13.6024|4.9274|        1.0|
|  Bream| 500.0|   26.8|   29.7|   34.5|14.1795|5.2785|        1.0|
|  Bream| 390.0|   27.6|   30.0|   35.0|  12.67|  4.69|        1.0|
|  Bream| 450.0|   27.6|   30.0|   35.1|14.0049|4.8438|        1.0|
|  Bream| 500.0|   28.5|   30.7|   36.2|14.2266|4.9594|        1.0|
|  Bream| 475.0|   28.4|   31.0|   36.2|14.2628|5.1042|        1.0|
|  Bream| 500.0|   28.7|   31.0|   36.2|14.3714|

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=[ 'Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width'], outputCol = "Features")

In [ ]:
assembler

VectorAssembler_bf0f32c5ac45

In [ ]:
output = assembler.transform(indexed)

In [ ]:
output.show(truncate = False)

+-------+------+-------+-------+-------+-------+------+-----------+-------------------------------------+
|Species|Weight|Length1|Length2|Length3|Height |Width |Species_cat|Features                             |
+-------+------+-------+-------+-------+-------+------+-----------+-------------------------------------+
|Bream  |242.0 |23.2   |25.4   |30.0   |11.52  |4.02  |1.0        |[242.0,23.2,25.4,30.0,11.52,4.02]    |
|Bream  |290.0 |24.0   |26.3   |31.2   |12.48  |4.3056|1.0        |[290.0,24.0,26.3,31.2,12.48,4.3056]  |
|Bream  |340.0 |23.9   |26.5   |31.1   |12.3778|4.6961|1.0        |[340.0,23.9,26.5,31.1,12.3778,4.6961]|
|Bream  |363.0 |26.3   |29.0   |33.5   |12.73  |4.4555|1.0        |[363.0,26.3,29.0,33.5,12.73,4.4555]  |
|Bream  |430.0 |26.5   |29.0   |34.0   |12.444 |5.134 |1.0        |[430.0,26.5,29.0,34.0,12.444,5.134]  |
|Bream  |450.0 |26.8   |29.7   |34.7   |13.6024|4.9274|1.0        |[450.0,26.8,29.7,34.7,13.6024,4.9274]|
|Bream  |500.0 |26.8   |29.7   |34.5   |14.179

In [ ]:
output.select("features","Species_cat").show(truncate = False)

+-------------------------------------+-----------+
|features                             |Species_cat|
+-------------------------------------+-----------+
|[242.0,23.2,25.4,30.0,11.52,4.02]    |1.0        |
|[290.0,24.0,26.3,31.2,12.48,4.3056]  |1.0        |
|[340.0,23.9,26.5,31.1,12.3778,4.6961]|1.0        |
|[363.0,26.3,29.0,33.5,12.73,4.4555]  |1.0        |
|[430.0,26.5,29.0,34.0,12.444,5.134]  |1.0        |
|[450.0,26.8,29.7,34.7,13.6024,4.9274]|1.0        |
|[500.0,26.8,29.7,34.5,14.1795,5.2785]|1.0        |
|[390.0,27.6,30.0,35.0,12.67,4.69]    |1.0        |
|[450.0,27.6,30.0,35.1,14.0049,4.8438]|1.0        |
|[500.0,28.5,30.7,36.2,14.2266,4.9594]|1.0        |
|[475.0,28.4,31.0,36.2,14.2628,5.1042]|1.0        |
|[500.0,28.7,31.0,36.2,14.3714,4.8146]|1.0        |
|[500.0,29.1,31.5,36.4,13.7592,4.368] |1.0        |
|[340.0,29.5,32.0,37.3,13.9129,5.0728]|1.0        |
|[600.0,29.4,32.0,37.2,14.9544,5.1708]|1.0        |
|[600.0,29.4,32.0,37.2,15.438,5.58]   |1.0        |
|[700.0,30.4

In [ ]:
final_data = output.select("features","Species_cat")

In [ ]:
final_data.show()

+--------------------+-----------+
|            features|Species_cat|
+--------------------+-----------+
|[242.0,23.2,25.4,...|        1.0|
|[290.0,24.0,26.3,...|        1.0|
|[340.0,23.9,26.5,...|        1.0|
|[363.0,26.3,29.0,...|        1.0|
|[430.0,26.5,29.0,...|        1.0|
|[450.0,26.8,29.7,...|        1.0|
|[500.0,26.8,29.7,...|        1.0|
|[390.0,27.6,30.0,...|        1.0|
|[450.0,27.6,30.0,...|        1.0|
|[500.0,28.5,30.7,...|        1.0|
|[475.0,28.4,31.0,...|        1.0|
|[500.0,28.7,31.0,...|        1.0|
|[500.0,29.1,31.5,...|        1.0|
|[340.0,29.5,32.0,...|        1.0|
|[600.0,29.4,32.0,...|        1.0|
|[600.0,29.4,32.0,...|        1.0|
|[700.0,30.4,33.0,...|        1.0|
|[700.0,30.4,33.0,...|        1.0|
|[610.0,30.9,33.5,...|        1.0|
|[650.0,31.0,33.5,...|        1.0|
+--------------------+-----------+
only showing top 20 rows



In [ ]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
train_data.show()

+--------------------+-----------+
|            features|Species_cat|
+--------------------+-----------+
|[0.0,19.0,20.5,22...|        2.0|
|[5.9,7.5,8.4,8.8,...|        0.0|
|[6.7,9.3,9.8,10.8...|        4.0|
|[7.0,10.1,10.6,11...|        4.0|
|[7.5,10.0,10.5,11...|        4.0|
|[8.7,10.8,11.3,12...|        4.0|
|[9.7,10.4,11.0,12...|        4.0|
|[9.8,10.7,11.2,12...|        4.0|
|[9.8,11.4,12.0,13...|        4.0|
|[10.0,11.3,11.8,1...|        4.0|
|[12.2,12.1,13.0,1...|        4.0|
|[13.4,11.7,12.4,1...|        4.0|
|[19.7,13.2,14.3,1...|        4.0|
|[32.0,12.5,13.7,1...|        0.0|
|[40.0,13.8,15.0,1...|        0.0|
|[55.0,13.5,14.7,1...|        5.0|
|[69.0,16.5,18.2,2...|        2.0|
|[70.0,15.7,17.4,1...|        0.0|
|[78.0,17.5,18.8,2...|        2.0|
|[80.0,17.2,19.0,2...|        0.0|
+--------------------+-----------+
only showing top 20 rows



In [ ]:
test_data.show()

+--------------------+-----------+
|            features|Species_cat|
+--------------------+-----------+
|[9.9,11.3,11.8,13...|        4.0|
|[12.2,11.5,12.2,1...|        4.0|
|[19.9,13.8,15.0,1...|        4.0|
|[40.0,12.9,14.1,1...|        2.0|
|[51.5,15.0,16.2,1...|        0.0|
|[60.0,14.3,15.5,1...|        5.0|
|[78.0,16.8,18.7,1...|        0.0|
|[85.0,17.8,19.6,2...|        0.0|
|[110.0,20.0,22.0,...|        0.0|
|[130.0,20.0,22.0,...|        0.0|
|[135.0,20.0,22.0,...|        0.0|
|[145.0,20.5,22.0,...|        2.0|
|[150.0,18.4,20.0,...|        5.0|
|[150.0,20.4,22.0,...|        2.0|
|[150.0,21.0,23.0,...|        0.0|
|[170.0,21.5,23.5,...|        0.0|
|[242.0,23.2,25.4,...|        1.0|
|[260.0,25.4,27.5,...|        0.0|
|[265.0,25.4,27.5,...|        0.0|
|[270.0,24.1,26.5,...|        6.0|
+--------------------+-----------+
only showing top 20 rows



In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
df_classifier = DecisionTreeClassifier(labelCol = "Species_cat").fit(train_data)

In [ ]:
df_predictions = df_classifier.transform(test_data)

In [ ]:
df_predictions.show()

+--------------------+-----------+--------------------+--------------------+----------+
|            features|Species_cat|       rawPrediction|         probability|prediction|
+--------------------+-----------+--------------------+--------------------+----------+
|[9.9,11.3,11.8,13...|        4.0|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       4.0|
|[12.2,11.5,12.2,1...|        4.0|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       4.0|
|[19.9,13.8,15.0,1...|        4.0|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       4.0|
|[40.0,12.9,14.1,1...|        2.0|[25.0,0.0,14.0,4....|[0.55555555555555...|       0.0|
|[51.5,15.0,16.2,1...|        0.0|[25.0,0.0,14.0,4....|[0.55555555555555...|       0.0|
|[60.0,14.3,15.5,1...|        5.0|[25.0,0.0,14.0,4....|[0.55555555555555...|       0.0|
|[78.0,16.8,18.7,1...|        0.0|[25.0,0.0,14.0,4....|[0.55555555555555...|       0.0|
|[85.0,17.8,19.6,2...|        0.0|[25.0,0.0,14.0,4....|[0.55555555555555...|       0.0|
|[110.0,20.0,22.0,...|        0.

In [ ]:
df_accuracy = MulticlassClassificationEvaluator(labelCol= "Species_cat",
                                                metricName="accuracy").evaluate(df_predictions)

In [ ]:
df_accuracy

0.7058823529411765